# Parte de Programación

Cargamos las principales librerías que vamos a utilizar. En este caso con **numpy** y **pandas**

In [1]:
import numpy as np
import pandas as pd

## Cargamos la base de datos

En este caso, se presentó un detalle de codificación de la base de datos desde la liga <https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow> por lo que se optó por descargar la página en formato **.json** y cargar el archivo desde una ubicación en la computadora

In [2]:
df = pd.read_json('../Python/search.json')
df.head()

,items,has_more,quota_max,quota_remaining
0,"{'tags': ['perl'], 'owner': {'reputation': 479...",True,300,298
1,"{'tags': ['file', 'perl', 'parameters'], 'owne...",True,300,298
2,"{'tags': ['powershell', 'perl', 'visual-studio...",True,300,298
3,"{'tags': ['perl', 'bareword'], 'owner': {'repu...",True,300,298
4,"{'tags': ['xml', 'perl', 'encoding'], 'owner':...",True,300,298


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   items            30 non-null     object
 1   has_more         30 non-null     bool  
 2   quota_max        30 non-null     int64 
 3   quota_remaining  30 non-null     int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 878.0+ bytes


Nos damos cuenta, accediendo a una entrada, de que no se puede leer un Data Frame directamente pues en las entradas de la columna **items** hay diccionarios. No sólo eso, estos diccionarios están compuestos por otro diccionario con la clave **owner**. Así que nos encontramos ante un caso de diccionarios anidados

## ¿Qué es lo que sigue entonces ?

Como se ha escogido usar pandas y no se puede obtener directamente un data frame, habrá que construir uno. Para ello se implementó un ciclo **for** que nos ayuda a acceder a cada clave de interés de los diccionarios. Así, se almacena el valor común de cada clave como una columna, y su valor se deposita en las celdas de los valores. Terminando con el método **append()** para su almacenaje.
En otras palabras, el código de la casilla siguiente hace lo siguiente:
1. Se escoge una llave  que será la columna de nuestro data frame, y se le asigna un valor nulo. Por ejemplo **user_id =[ ]**
2. Se llama a un ciclo **for** que acceda a todos los elementos  de la columna **items** y tome la llave de interés, **user_id**.
3. Finalmente, se almacena en una lista con el método **append()**

In [4]:
user_id=[]
is_answered=[]
display_name= []
view_count = []
answer_count = []
user_type= []
reputation = []
last_activity_date = []
creation_date = []
title = []
question_id=[]
for i in range(0,30):
    a=df['items'][i]['owner']['user_id']
    user_id.append(a)
    
    b=df['items'][i]['owner']['display_name']
    display_name.append(b)
    
    c=df['items'][i]['view_count']
    view_count.append(c) 
    
    d=df['items'][i]['is_answered']
    is_answered.append(d)
    
    e=df['items'][i]['answer_count']
    answer_count.append(e)
    
    f=df['items'][i]['owner']['user_type']
    user_type.append(f)
    
    g= df['items'][i]['owner']['reputation']
    reputation.append(g)
    
    h= df['items'][i]['last_activity_date']
    last_activity_date.append(h)
    
    j= df['items'][i]['creation_date']
    creation_date.append(j)
    
    k= df['items'][i]['question_id']
    question_id.append(k)
    
    m= df['items'][i]['title']
    title.append(m)

Posteriormente se crean los Data Frames en donde la llave del diccionario servirá como el identificador de la columna

In [5]:
df_user_id = pd.DataFrame(user_id, columns = ['user_id'])
df_is_answered = pd.DataFrame(is_answered, columns = ['is_answered'])
df_display_name = pd.DataFrame(display_name, columns = ['Name'])
df_view_count = pd.DataFrame(view_count, columns = ['view_count'])
df_answer_count = pd.DataFrame(answer_count, columns = ['answer_count'])
df_user_type = pd.DataFrame(user_type, columns = ['user_type'])
df_reputation = pd.DataFrame(reputation, columns = ['reputation'])
df_last_activity_date = pd.DataFrame(last_activity_date, columns = ['last_activity_date'])
df_creation_date = pd.DataFrame(creation_date, columns = ['creation_date'])
df_question_id = pd.DataFrame(question_id, columns = ['question_id'])
df_title = pd.DataFrame(title, columns = ['title'])

De un análisis preliminar de los datos se puede notar que los datos concernientes a fechas no están en un formato legible a simple vista por el usuario. Estos se hallan en formato **epoch** por lo que habrá que convertirlos a un formato de fecha y hora. Esto lo conseguimos importando el módulo **datetime**

In [6]:
import datetime

Ahora habrá que acceder a cada valor de las columnas **last_activity_date** y **creation_date** y convertir los datos de tipo epoch a una fecha convencional. Esto se consigue mediante un ciclo **for** en el que a cada dato se le aplica la conversión **datetime.datetime.fromtimestamp(epoch_time)**. Así obtendremos dos listas, con los identificadores correspondientes, de datos con fechas legibles.

In [7]:
last_activity_date = []
creation_date = []
for i in range(0,30):
    epoch_time1 = df_last_activity_date['last_activity_date'][i]
    last_date= datetime.datetime.fromtimestamp(epoch_time1)
    last_activity_date.append(last_date)
    
    epoch_time2 = df_creation_date['creation_date'][i]
    creation= datetime.datetime.fromtimestamp(epoch_time2)
    creation_date.append(creation)

Las listas que se obtuvieron en el paso anterior se transformarán ahora a Data Frames, justo como se hizo con anterioridad.

In [8]:
df_last_activity_date_2 = pd.DataFrame(last_activity_date, columns = ['last_activity_date'])
df_creation_date_2= pd.DataFrame(creation_date, columns = ['creation_date'])

Ya con los Data Frames creados, los podemos concatenar para tener un sólo Data Frame y se pueda acceder sencillamente a la información.

In [9]:
df1=pd.concat([df_display_name, df_user_id, df_is_answered, 
               df_view_count, df_answer_count, df_user_type,
               df_reputation,df_last_activity_date_2, df_creation_date_2,
               df_question_id, df_title ], axis = 1)
df1.head()

,Name,user_id,is_answered,view_count,answer_count,user_type,reputation,last_activity_date,creation_date,question_id,title
0,Lee Robinson,7061612,True,21,1,registered,479,2022-03-09 13:34:50,2022-03-09 12:27:11,71414292,How to iterate through an array of hashes insi...
1,Saratchandra MV,15394891,False,31,0,registered,11,2022-03-09 10:17:43,2022-03-09 08:11:48,71410702,How can I send input from Perl Script to Param...
2,nck,7019069,True,52,2,registered,1181,2022-03-09 09:41:03,2022-03-07 12:05:00,71385438,__FILE__ in perl returns paths without disk le...
3,n.r.,1839508,True,41,1,registered,1828,2022-03-09 08:05:26,2022-03-09 06:38:30,71409496,Why is a bareword following the concatenation ...
4,giordano,1236569,True,56,1,registered,2557,2022-03-09 04:41:10,2022-01-07 08:49:40,70622981,Perl: Repair utf8 xml file which contains octa...


# Sin embargo hay un pequeño problema !!!!

Un análisis más profundo nos deja ver que hay llaves de más en algunos elementos. En este caso **accepted_answer_id**.

In [10]:
df['items'][3]

{'tags': ['perl', 'bareword'],
 'owner': {'reputation': 1828,
  'user_id': 1839508,
  'user_type': 'registered',
  'accept_rate': 89,
  'profile_image': 'https://www.gravatar.com/avatar/530d84518abcb1591e5e7cedf682097d?s=256&d=identicon&r=PG',
  'display_name': 'n.r.',
  'link': 'https://stackoverflow.com/users/1839508/n-r'},
 'is_answered': True,
 'view_count': 41,
 'accepted_answer_id': 71410613,
 'answer_count': 1,
 'score': 4,
 'last_activity_date': 1646834726,
 'creation_date': 1646829510,
 'question_id': 71409496,
 'content_license': 'CC BY-SA 4.0',
 'link': 'https://stackoverflow.com/questions/71409496/why-is-a-bareword-following-the-concatenation-operator-a-string-in-perl',
 'title': 'Why is a bareword following the concatenation operator a string in Perl?'}

# Así que habrá que hacer una limpieza de datos más !

La llave anterior nos dice cuál es el **id** de la respuesta que se ha dado. Cómo no se tiene un registro, pero se desea agregar esos valores al Data Frame, habrá que establecer un valor "adecuado para estos valores faltantes". Se podrían interpretar como que no se respondió, así que se rellenan con el número **cero**. Para hacer esto, se explota el hecho de que los elementos de **item** son diccionarios y se uda la función **.setdefault()**. Cómo se va a aplicar a cada elemento de **items** que no tenga en sí a la llave llamada **accepted_answer_id**, se aplica un ciclo **for** a cada entrada y se convierten a cero los que no tengan esta llave.

In [11]:
for i in range (0,30):
    df['items'][i].setdefault('accepted_answer_id', 0)
    print(df['items'][i]['accepted_answer_id'])

0
0
0
71410613
70623627
71405937
71402827
0
0
71391815
0
12537699
0
0
71385077
11797737
71373450
0
3609021
71365666
0
59555238
71349788
71304629
71337274
0
0
0
0
71312890


Ahora habrá que hacer el Data Frame de esto:

In [12]:
accepted_answer_id=[]
for i in range(0,30):
    l=df['items'][i]['accepted_answer_id']
    accepted_answer_id.append(l)


In [13]:
df_accepted_answer_id = pd.DataFrame(accepted_answer_id, columns = ['accepted_answer_id'])
df_accepted_answer_id.head()

,accepted_answer_id
0,0
1,0
2,0
3,71410613
4,70623627


# Listo !
Con el Data Frame ya creado, lo concatenamos al Data Frame general de donde vamos a obtener toda nuestra información.

In [14]:
df1=pd.concat([df_user_id, df_user_type, df_display_name, df_reputation, df_title ,df_question_id, df_is_answered, 
               df_view_count, df_answer_count,
               df_creation_date_2, df_last_activity_date_2,
                df_accepted_answer_id,   ], axis = 1)
df1.head()

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes insi...,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0
1,15394891,registered,Saratchandra MV,11,How can I send input from Perl Script to Param...,71410702,False,31,0,2022-03-09 08:11:48,2022-03-09 10:17:43,0
2,7019069,registered,nck,1181,__FILE__ in perl returns paths without disk le...,71385438,True,52,2,2022-03-07 12:05:00,2022-03-09 09:41:03,0
3,1839508,registered,n.r.,1828,Why is a bareword following the concatenation ...,71409496,True,41,1,2022-03-09 06:38:30,2022-03-09 08:05:26,71410613
4,1236569,registered,giordano,2557,Perl: Repair utf8 xml file which contains octa...,70622981,True,56,1,2022-01-07 08:49:40,2022-03-09 04:41:10,70623627


# Pero podemos hacer el Data Frame más vistoso si a cada usuario le agregamos su foto de perfil !

Así, importamos primero la librería **HTML**.

In [15]:
from IPython.core.display import HTML

Creamos un Data Frame de los links de las fotos de perfil

In [16]:
profile_image=[]
for i in range(0,30):
    abc =df['items'][i]['owner']['profile_image']
    profile_image.append(abc)

# Listo !

In [17]:
df_profile_image = pd.DataFrame(profile_image, columns= ['profile_image'])
df_profile_image.head()

,profile_image
0,https://www.gravatar.com/avatar/e4c4482be2d3df...
1,https://lh6.googleusercontent.com/-UiLLOi86vGY...
2,https://i.stack.imgur.com/SQoGB.jpg?s=256&g=1
3,https://www.gravatar.com/avatar/530d84518abcb1...
4,https://www.gravatar.com/avatar/f0f37797600290...


In [18]:
df3=pd.concat([df1,df_profile_image ], axis=1)
df3.head()

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes insi...,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0,https://www.gravatar.com/avatar/e4c4482be2d3df...
1,15394891,registered,Saratchandra MV,11,How can I send input from Perl Script to Param...,71410702,False,31,0,2022-03-09 08:11:48,2022-03-09 10:17:43,0,https://lh6.googleusercontent.com/-UiLLOi86vGY...
2,7019069,registered,nck,1181,__FILE__ in perl returns paths without disk le...,71385438,True,52,2,2022-03-07 12:05:00,2022-03-09 09:41:03,0,https://i.stack.imgur.com/SQoGB.jpg?s=256&g=1
3,1839508,registered,n.r.,1828,Why is a bareword following the concatenation ...,71409496,True,41,1,2022-03-09 06:38:30,2022-03-09 08:05:26,71410613,https://www.gravatar.com/avatar/530d84518abcb1...
4,1236569,registered,giordano,2557,Perl: Repair utf8 xml file which contains octa...,70622981,True,56,1,2022-01-07 08:49:40,2022-03-09 04:41:10,70623627,https://www.gravatar.com/avatar/f0f37797600290...


Construímos la función que convierte la ruta de la imagen a un tag de **HTML**

In [19]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [20]:
df3.to_html(escape=False, formatters=dict(profile_image=path_to_image_html))

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>user_id</th>\n      <th>user_type</th>\n      <th>Name</th>\n      <th>reputation</th>\n      <th>title</th>\n      <th>question_id</th>\n      <th>is_answered</th>\n      <th>view_count</th>\n      <th>answer_count</th>\n      <th>creation_date</th>\n      <th>last_activity_date</th>\n      <th>accepted_answer_id</th>\n      <th>profile_image</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>7061612</td>\n      <td>registered</td>\n      <td>Lee Robinson</td>\n      <td>479</td>\n      <td>How to iterate through an array of hashes inside an object with perl</td>\n      <td>71414292</td>\n      <td>True</td>\n      <td>21</td>\n      <td>1</td>\n      <td>2022-03-09 12:27:11</td>\n      <td>2022-03-09 13:34:50</td>\n      <td>0</td>\n      <td><img src="https://www.gravatar.com/avatar/e4c4482be2d3dff4af610760684a05f1?s=256&d=identicon&r=PG&f=1" w

Renderizamos el Data Frame a una tabla **HTML** y finalmente llamamos al método **HTML** para moestrar las imágenes renderizadas.

# Y así tenemos una tabla más vistosa !!!

In [21]:
df4=HTML(df3.to_html(escape=False,formatters=dict(profile_image=path_to_image_html)))
df4

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes inside an object with perl,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0,
1,15394891,registered,Saratchandra MV,11,How can I send input from Perl Script to Parameter file (.prm),71410702,False,31,0,2022-03-09 08:11:48,2022-03-09 10:17:43,0,
2,7019069,registered,nck,1181,__FILE__ in perl returns paths without disk letter in visual studio code powershell,71385438,True,52,2,2022-03-07 12:05:00,2022-03-09 09:41:03,0,
3,1839508,registered,n.r.,1828,Why is a bareword following the concatenation operator a string in Perl?,71409496,True,41,1,2022-03-09 06:38:30,2022-03-09 08:05:26,71410613,
4,1236569,registered,giordano,2557,Perl: Repair utf8 xml file which contains octal or hexadecimal codes,70622981,True,56,1,2022-01-07 08:49:40,2022-03-09 04:41:10,70623627,
5,11297512,registered,Dschungel Ratte,35,perl replace with bash variable,71405507,True,51,1,2022-03-09 01:07:56,2022-03-09 02:02:16,71405937,
6,385390,registered,bliako,918,Perl XS: create and return array of strings (char*) taken from calling a C function or undef on failure,71402095,True,39,1,2022-03-08 16:12:44,2022-03-08 18:47:02,71402827,
7,16490939,registered,FlyingZeppo,15,Passing variable from PHP to perl is only reading first word,71357476,True,45,2,2022-03-04 15:39:47,2022-03-08 13:32:17,0,
8,18119587,registered,Vijayanand Balu,21,Compare 3 digit dot separated number with >3 digit and alphanumeric in Perl,70990227,True,66,1,2022-02-04 11:08:04,2022-03-08 13:17:50,0,
9,1641112,registered,StevieD,3627,Inline::Perl5 not working with Lingua::En::Titlecase perl 5 module,71391287,True,29,1,2022-03-08 01:17:37,2022-03-08 02:12:10,71391815,


# Pero !
No es tan facil acceder a los datos como en un Data Frame. Así que regresamos a nuestro Data Frame sin las imágenes

In [22]:
df3.head()

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes insi...,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0,https://www.gravatar.com/avatar/e4c4482be2d3df...
1,15394891,registered,Saratchandra MV,11,How can I send input from Perl Script to Param...,71410702,False,31,0,2022-03-09 08:11:48,2022-03-09 10:17:43,0,https://lh6.googleusercontent.com/-UiLLOi86vGY...
2,7019069,registered,nck,1181,__FILE__ in perl returns paths without disk le...,71385438,True,52,2,2022-03-07 12:05:00,2022-03-09 09:41:03,0,https://i.stack.imgur.com/SQoGB.jpg?s=256&g=1
3,1839508,registered,n.r.,1828,Why is a bareword following the concatenation ...,71409496,True,41,1,2022-03-09 06:38:30,2022-03-09 08:05:26,71410613,https://www.gravatar.com/avatar/530d84518abcb1...
4,1236569,registered,giordano,2557,Perl: Repair utf8 xml file which contains octa...,70622981,True,56,1,2022-01-07 08:49:40,2022-03-09 04:41:10,70623627,https://www.gravatar.com/avatar/f0f37797600290...


# Ahora vamos a hacer las instrucciones que se pidieron para este reto.

##  Número de respuestas contestadas y no contestadas

Primeramente argrupamos los datos y usamos la función **.count()** para que podamos contar el número de **True** y **False** que tenemos en nuestro Data Frame.

In [23]:
NEW = df3.groupby('is_answered')['is_answered'].count()
NEW

is_answered
False     5
True     25
Name: is_answered, dtype: int64

### Tenemos por lo tanto que se respondieron 25 preguntas de 30 y el resto quedaron sin responder

# Pregunta con el menor número de vistas 

Para saber cuál fue la pregunta con el menor número de vistas, usamos la función **.nsmallest( )** y por curiosisdad nos enfocamos en las primeras 4

In [24]:
df3.nsmallest(4, 'view_count')

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
27,18362277,registered,user18362277,1,Use Perl GIT Module without installing it in L...,71334361,False,19,0,2022-03-03 02:52:55,2022-03-03 03:09:31,0,https://www.gravatar.com/avatar/9c930eeea5ad0b...
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes insi...,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0,https://www.gravatar.com/avatar/e4c4482be2d3df...
13,13087546,registered,hani craft,1,call a perl script from python gui programming,71385063,False,21,0,2022-03-07 11:33:33,2022-03-07 11:47:57,0,https://lh3.googleusercontent.com/a-/AOh14GhUg...
9,1641112,registered,StevieD,3627,Inline::Perl5 not working with Lingua::En::Tit...,71391287,True,29,1,2022-03-08 01:17:37,2022-03-08 02:12:10,71391815,https://www.gravatar.com/avatar/00bee18fb974a0...


Podemos concluir que la pregunta que recibió menos vistas fue la que hizo **user18362277**, con reputación **1** y cuya pregunta fue:

In [25]:
df3['title'][27]

'Use Perl GIT Module without installing it in Linux Box'

# Respuesta más vieja

In [26]:
df3.nsmallest(1, 'last_activity_date')

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
29,7880079,registered,Kousik Mandal,654,perl DBI module installation erroring out,71308678,True,81,2,2022-03-01 06:43:00,2022-03-02 07:45:40,71312890,https://www.gravatar.com/avatar/fd66adfb431ad8...


Se vuelve a usar la función **.nsmallest( )** y la fecha más vieja es:

In [27]:
df3['last_activity_date'][29]

Timestamp('2022-03-02 07:45:40')

# Respuesta más actual

Se usa ahora la función **.nlargest()**

In [28]:
df3.nlargest(2, 'last_activity_date')

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
0,7061612,registered,Lee Robinson,479,How to iterate through an array of hashes insi...,71414292,True,21,1,2022-03-09 12:27:11,2022-03-09 13:34:50,0,https://www.gravatar.com/avatar/e4c4482be2d3df...
1,15394891,registered,Saratchandra MV,11,How can I send input from Perl Script to Param...,71410702,False,31,0,2022-03-09 08:11:48,2022-03-09 10:17:43,0,https://lh6.googleusercontent.com/-UiLLOi86vGY...


Siendo la más actual :

In [29]:
df3['last_activity_date'][0]

Timestamp('2022-03-09 13:34:50')

# Finalmente la pregunta del owner con una mayor reputación se obtiene

Nuevamente se usa la función **nlargest** esta vez enfocada en la columna **reputation**

In [30]:
df3.nlargest(3, 'reputation')

,user_id,user_type,Name,reputation,title,question_id,is_answered,view_count,answer_count,creation_date,last_activity_date,accepted_answer_id,profile_image
18,113124,registered,Lazer,84271,What are the common workarounds for multi-line...,3608957,True,9638,13,2010-08-31 07:23:47,2022-03-06 08:30:58,3609021,https://www.gravatar.com/avatar/77d0f6949f176d...
24,180275,registered,Ren&#233; Nyffenegger,37464,How can I use Perl for in-place-editing of fil...,71335069,True,50,1,2022-03-03 03:48:01,2022-03-03 18:04:07,71337274,https://i.stack.imgur.com/4GALY.jpg?s=256&g=1
17,182693,registered,Sean,28026,Returning values from exception handlers in Pe...,51644197,True,471,7,2018-08-01 20:07:47,2022-03-06 11:13:34,0,https://www.gravatar.com/avatar/7e4422dd860fce...


Así, el usuario con la mayor reputación es **Lazer** y su pregunta fue :

In [31]:
df3['title'][18]

'What are the common workarounds for multi-line comments in Perl?'